# Hyperparameters search

In past notebooks, we pointing out that some models' parameters had an impact on the statistical performance of the models. Usually, we would like to optimize these parameters such that the model trained is as good as possible. This optimization is called hyperparameters tuning.

In this notebook, we will show a couple of method allowing to tune models' hyperparameters.

## Introductory example

We will take an example that we showed in the linear model where we discussed the impact of the $\alpha$ parameter on a `Ridge` model. Indeed, we mentioned that this parameter allows to regularize more or less the model. However, there is no general rule specifying what is a good $\alpha$ value. Indeed, it would depend of the dataset.

Let's load a dataset to tackle a regression problem.

In [ ]:
from sklearn.datasets import fetch_california_housing

X, y = fetch_california_housing(return_X_y=True, as_frame=True)
X.head()

In [ ]:
y.head()

Now, we will define a `Ridge` model where we will process the the data with add some interaction between features using a `PolynomialFeatures` transformer.

In [ ]:
import sklearn

sklearn.set_config(display="diagram")

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline

model = make_pipeline(
    PolynomialFeatures(),
    StandardScaler(),
    Ridge(),
)
model

However, we did not change any of the default parameters given by scikit-learn. Let's evaluate this vanilla model.

In [ ]:
import pandas as pd
from sklearn.model_selection import cross_validate

cv_results = cross_validate(model, X, y)
cv_results = pd.DataFrame(cv_results)

In [ ]:
cv_results

In [ ]:
cv_results.aggregate(["mean", "std"])

At this stage, there is nothing to tell use that our pipeline is the best pipeline that we could get. Indeed, we could imagine that the degree of the `PolynomialFeatures` could be higher or that the `Ridge` regressor should be more regularized. Let's check which parameters we could tune with the model:

In [ ]:
for params in model.get_params():
    print(params)

Two important parameters of this model are `polynomialfeatures__degree` and `ridge__alpha`. We will try to find the optimal values of these parameters for the current dataset.

## Manual hyperparameters search

Before to show the automated tools allowing to make hyperparameters tuning in scikit-learn, we will manually make our own manual simplified version.

<div class="alert alert-success">
    <b>EXERCISE</b>:
    <ul>
        <li>Split the dataset into a training and testing set.</li>
        <li>Make a nested <tt>for</tt> loop to try all the possible parameters combination that we defined in <tt>parameter_grid</tt>.</li>
        <li>In the internal loop, use a cross-validation (using <tt>cross_val_score</tt>) on the training set to get a distribution of score.</li>
        <li>Compute the mean and standard deviation of the cross-validation score and pick-up the best set of hyperparameters.</li>
        <li>Retrain a model with the combination of the best hyperparameters and evalute it on the testing set.</li>
    </ul>
</div>

In [ ]:
# %load solutions/solution_01.py

In [ ]:
# %load solutions/solution_02.py

In [ ]:
# %load solutions/solution_03.py

In [ ]:
# %load solutions/solution_04.py

In [ ]:
# %load solutions/solution_05.py

In [ ]:
# %load solutions/solution_06.py

In [ ]:
# %load solutions/solution_07.py

## Hyperparameters search using a grid

The search that we performed manually is indeed known as a grid-search: we try every possible combination of the parameter that we first provided. Scikit-learn provides a specific estimator that will make the processing that we did previously: during `fit`, it will perform a cross-validation and pick the optimal hyperparameters using cross-validation.

In [ ]:
from sklearn.model_selection import GridSearchCV

search_cv = GridSearchCV(model, param_grid=parameter_grid)
search_cv.fit(X_train, y_train)

We can get the best found parameters by looking at the fitted attributes `best_params_`:

In [ ]:
search_cv.best_params_

We can even get more information regarding the different combinations of hyperparameters tried during `fit` by looking at the fitted attribute `cv_results_`:

In [ ]:
cv_results = pd.DataFrame(search_cv.cv_results_)
cv_results.head()

In addition, at the end of the `fit` procedure, if the parameter `refit` is set to `True` (default), a model with the best combination will be trained (as we did in the manual hyperparameters search). We can check this model by looking at the fitted attribute `best_estimator_`

In [ ]:
search_cv.best_estimator_

Indeed, this `best_estimator_` will be used when calling `predict` and `score` from the `GridSearchCV` instance.

In [ ]:
search_cv.score(X_test, y_test)

<div class="alert alert-success">
    <b>EXERCISE</b>:
    <br>
    Since a <tt>GridSearchCV</tt> behave like any classifier or regressor, it can be used and evaluated by cross-validation. Use <tt>cross_validate</tt> to evaluate the previous grid-search model that we created.
</div>

In [ ]:
# %load solutions/solution_08.py

In [ ]:
# %load solutions/solution_09.py

In [ ]:
# %load solutions/solution_10.py

<div class="alert alert-success">
    <b>QUESTION</b>:
    <br>
    Which limitation the grid-search approach suffer from?
</div>

## Randomized hyperparameters search

In the previous strategy, the grid-search has two limitations:

- it only explores combination of parameters defined in the grid;
- adding new parameters and values to explore will increase rapidly the cost of the search.

`RandomizedSearchCV` allows to specify a distribution from which to draw parameter values. It allows to explore the hyperparamters space on a non-grid fashion and as a user, you can give a butget of the number of combination you want to try.

In [ ]:
from scipy.stats import loguniform

parameter_distributions = {
    "polynomialfeatures__degree": np.arange(1, 5),
    "ridge__alpha": loguniform(1, 3),
}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

search_cv = RandomizedSearchCV(
    model, param_distributions=parameter_distributions, n_iter=10,
)

In [ ]:
cv_results = cross_validate(search_cv, X, y, return_estimator=True)
cv_results = pd.DataFrame(cv_results)

In [ ]:
cv_results

In [ ]:
for est in cv_results["estimator"]:
    print(est.best_params_)

## Model with internal hyperparameter tuning

Some classifiers or regressors come with the some efficient hyperparameter selection, at least more efficient than a grid-search. Usually, the name of the classsifiers or regressors finish with `CV` (e.g. `RidgeCV`).

<div class="alert alert-success">
    <b>EXERCISE</b>:
    <br>
    <ul>
        <li>Create a pipeline made of a <tt>PolynomialFeatures</tt>, a <tt>StandardScaler</tt>, and a <tt>Ridge</tt>.</li>
        <li>Create a grid-search by passing the previous pipeline and tune the parameter <tt>alpha</tt> such that you will try the values <tt>np.logspace(-2, 2, num=50)</tt>.</li>
        <li>Fit the grid-search on the training set and check the time it takes.</li>
        <li>Repeat the experiment by replacing the <tt>Ridge</tt> regressor by a <tt>RidgeCV</tt> regressor and removing the <tt>GridSearchCV</tt>.</li>
    </ul>
    Which approach is more efficient in terms of computational performance.
</div>

In [ ]:
# %load solutions/solution_11.py

In [ ]:
# %load solutions/solution_12.py

In [ ]:
# %load solutions/solution_13.py

In [ ]:
# %load solutions/solution_14.py

## Inspection of hyperparameters in cross-validation

Sometimes, we perform a search cross-validation inside a cross-validation evaluation. In this case, we potentially have different set of hyperparameter values for each individual cross-validation split. We can indeed inspect these values. Let's define a `GridSearchCV` model.

In [ ]:
from sklearn.linear_model import Ridge

alphas = np.logspace(-2, 2, num=50)

model = GridSearchCV(
    make_pipeline(
        PolynomialFeatures(),
        StandardScaler(),
        Ridge(),
    ),
    param_grid={
        "ridge__alpha": alphas
    },
    scoring="neg_mean_squared_error",
)
model

Then, we can run a cross-validation by passing the model to `cross_validate`. In addition, we can store every model train on each cross-validation splits by setting `return_estimator` to `True`.

In [ ]:
cv_results = cross_validate(model, X, y, cv=3, return_estimator=True)
cv_results = pd.DataFrame(cv_results)
cv_results

We see that the `estimator` columns contain the different estimators. Thus we can check the `best_params_` stored by the `GridSearchCV`.

In [ ]:
for estimator_cv_fold in cv_results["estimator"]:
    print(estimator_cv_fold.best_params_)

Such inspection allows to study the stability of the hyperparameter values.